In [1]:
#get data on ways fighter won/lost through dec, won/lost through sub, won/lost through ko/tko  
#tabulate outcomes in a table such as: Outcome, % chance, winner 
#FRONT END: give the user the ability to input two fighters (drop down menu)
# ---- FINISHED --- 
#extra: prevent users from picking two fighters that are from different weight classes
#--- LONG RUN --- 
# Improve model where it takes # of strikes (successful, attempted) , ground game/submissions duration, takedowns (successful, attempted)
# into consideration 

In [2]:
import pandas as pd 

In [3]:
df = pd.read_csv("UFC Matches April 2023.csv")

In [4]:
df.head()

,Full Name,Weight,Matchup,Outcome,Method,Round,Time
0,Tom Aaron,WEIGHT: 155 lbs.,NaN,NaN,NaN,NaN,NaN
1,Tom Aaron,WEIGHT: 155 lbs.,Tom Aaron\n \n Matt Ricehouse,LOSS,U-DEC,3.0,5:00
2,Tom Aaron,WEIGHT: 155 lbs.,Tom Aaron\n \n Eric Steenberg,WIN,SUB,1.0,0:56
3,Danny Abbadi,WEIGHT: 155 lbs.,NaN,NaN,NaN,NaN,NaN
4,Danny Abbadi,WEIGHT: 155 lbs.,Danny Abbadi\n \n Jorge Gurgel,LOSS,S-DEC,3.0,5:00


## Data Cleaning

We have to clean our dataset then group each fighter and get their distributions to determine their likelihood of winning/losing via decision, tko/ko or submission.

In [5]:
# delete first row of every fighter
nulls = df[["Matchup", "Outcome", 'Method', 'Round', 'Time']].isnull().all(axis=1) 
df = df.loc[~nulls, :]

#find matches that are upcoming and delete them
df = df[df["Outcome"] != 'NEXT']

# delete "weight text" in weight column
df['Weight'] = df['Weight'].str.replace('WEIGHT: ', '')

# keep only opponents name in matchup column
def remove_name(orig_matchup):
    last_index = orig_matchup.rfind('\n')
    new_string = orig_matchup[last_index +1:]
    return new_string.strip()

df['Matchup'] = df['Matchup'].apply(lambda x: remove_name(x))

In [6]:
unique_methods = df['Method'].unique()

In [7]:
unique_methods

array(['U-DEC', 'SUB', 'S-DEC', 'KO/TKO', 'Overturned', 'Other', 'M-DEC',
       'CNC', 'DQ', 'SUB ', 'KO/TKO ', 'S-DEC ', 'Decision', 'U-DEC ',
       'M-DEC ', 'Overturned ', 'CNC '], dtype=object)

In [8]:
unique_outcomes = df['Outcome'].unique()

In [9]:
unique_outcomes

array(['LOSS', 'WIN', 'NC', 'DRAW'], dtype=object)

In [10]:
removed_outcomes = ['NC', 'DRAW']
removed_methods = ['Overturned', 'Other', 'CNC', 'DQ', 'Overturned ', 'CNC ']

In [11]:
filtered_df1 = df.loc[~df['Method'].isin(removed_methods)]
filtered_df2 = filtered_df1.loc[~df['Outcome'].isin(removed_outcomes)]
filtered_df2['Method'] = filtered_df2['Method'].apply(lambda x: x.strip())

<ipython-input-11-fcbd382802ee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df2['Method'] = filtered_df2['Method'].apply(lambda x: x.strip())


In [12]:
df = filtered_df2

## Exploratory Data Analysis (EDA)

In [13]:
decisions = ['U-DEC', 'S-DEC', 'M-DEC', 'Decision']
kos = ['KO/TKO']
submissions = ['SUB']

In [14]:
decision_df = df[df['Method'].isin(decisions)]
decision_count = decision_df['Method'].count()

ko_df = df[df['Method'].isin(kos)]
ko_count = ko_df['Method'].count()

submission_df = df[df['Method'].isin(submissions)]
submission_count = submission_df['Method'].count()

print(decision_count/len(df))
print(ko_count/len(df))
print(submission_count/len(df))

0.428926871484206
0.34714409346603203
0.223929035049762


In [15]:
#make multipliers?
#0.4286023366508005 --> 43% end by decision
#0.34714409346603203 --> 35% end by ko
#0.223929035049762 --> 22% end by submission

In [16]:
#questions that could be of importance:
#what is the most common method of winning?
#for fighters that win through ko/submission what round does it typically occur in?
#is there a discrepancy in the way fighters win across different weight classes?

In [17]:
def transform_dataset(df):
    data = {
    'Total Fights': df.groupby(['Full Name'])['Matchup'].count(),
    'Total Wins': df.loc[df['Outcome'] == 'WIN'].groupby(['Full Name'])['Outcome'].count(),
    'Total Losses': df.loc[df['Outcome'] == 'LOSS'].groupby(['Full Name'])['Outcome'].count(),
    'Wins By Decision': df.loc[(df['Outcome'] == 'WIN') & (df['Method'].isin(decisions))].groupby(['Full Name'])['Outcome'].count(),
    'Wins By KO': df.loc[(df['Outcome'] == 'WIN') & (df['Method'].isin(kos))].groupby(['Full Name'])['Outcome'].count(),
    'Wins By Submission': df.loc[(df['Outcome'] == 'WIN') & (df['Method'].isin(submissions))].groupby(['Full Name'])['Outcome'].count(),
    'Loss By Decision': df.loc[(df['Outcome'] == 'LOSS') & (df['Method'].isin(decisions))].groupby(['Full Name'])['Outcome'].count(),
    'Loss By KO': df.loc[(df['Outcome'] == 'LOSS') & (df['Method'].isin(kos))].groupby(['Full Name'])['Outcome'].count(),
    'Loss By Submission': df.loc[(df['Outcome'] == 'LOSS') & (df['Method'].isin(submissions))].groupby(['Full Name'])['Outcome'].count()
}
    fighter_stats = pd.DataFrame(data)
    fighter_stats.fillna(0, inplace=True)
    return fighter_stats

In [18]:
df = transform_dataset(df)

In [19]:
df.head()

,Total Fights,Total Wins,Total Losses,Wins By Decision,Wins By KO,Wins By Submission,Loss By Decision,Loss By KO,Loss By Submission
AJ Dobson,3,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0
AJ Fletcher,4,2.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0
AJ Fonseca,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
AJ Matthews,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
Aalon Cruz,3,1.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0


## Monte Carlo Simulation

In [20]:
import math
import random as rnd

In [21]:
def calculate_mean(statistic, total_fights):
    return statistic/total_fights

def calculate_sd(statistic, total_fights):
    mean = calculate_mean(statistic, total_fights)
    return math.sqrt((mean*(1-mean))/total_fights)

In [22]:
def get_fighter_parameters(fighter1, fighter2):
    data = {"Decision Wins Mean" : [calculate_mean(df.loc[fighter1, "Wins By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Wins By Decision"], df.loc[fighter2, "Total Fights"])], 
            
           "Decision Wins SD" : [calculate_sd(df.loc[fighter1, "Wins By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Wins By Decision"], df.loc[fighter2, "Total Fights"])],
            
           "KO Wins Mean" : [calculate_mean(df.loc[fighter1, "Wins By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Wins By KO"], df.loc[fighter2, "Total Fights"])], 
            
            'KO Wins SD' : [calculate_sd(df.loc[fighter1, "Wins By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Wins By KO"], df.loc[fighter2, "Total Fights"])], 
           
            'Sub Wins Mean' : [calculate_mean(df.loc[fighter1, "Wins By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Wins By Submission"], df.loc[fighter2, "Total Fights"])],
           
            'Sub Wins SD' : [calculate_sd(df.loc[fighter1, "Wins By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Wins By Submission"], df.loc[fighter2, "Total Fights"])], 
            
            'Decision Loss Mean' : [calculate_mean(df.loc[fighter1, "Loss By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Loss By Decision"], df.loc[fighter2, "Total Fights"])], 
            
            'Decision Loss SD' : [calculate_sd(df.loc[fighter1, "Loss By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Loss By Decision"], df.loc[fighter2, "Total Fights"])], 
            
            'KO Loss Mean' : [calculate_mean(df.loc[fighter1, "Loss By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Loss By KO"], df.loc[fighter2, "Total Fights"])], 
            
            'KO Loss SD' : [calculate_sd(df.loc[fighter1, "Loss By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Loss By KO"], df.loc[fighter2, "Total Fights"])], 
           
            'Sub Loss Mean' : [calculate_mean(df.loc[fighter1, "Loss By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_mean(df.loc[fighter2, "Loss By Submission"], df.loc[fighter2, "Total Fights"])], 
            
            'Sub Loss SD' : [calculate_sd(df.loc[fighter1, "Loss By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Loss By Submission"], df.loc[fighter2, "Total Fights"])]}
    
    fighter_parameter_df = pd.DataFrame(data=data, index = [fighter1, fighter2])
    return fighter_parameter_df

In [23]:
def gameSim():
    results = []
    fighter1_dec_score = (rnd.gauss(matchup_df.iloc[0]['Decision Wins Mean'],matchup_df.iloc[0]['Decision Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[1]['Decision Loss Mean'],matchup_df.iloc[1]['Decision Loss SD']))/2
    fighter1_ko_score = (rnd.gauss(matchup_df.iloc[0]['KO Wins Mean'],matchup_df.iloc[0]['KO Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[1]['KO Loss Mean'],matchup_df.iloc[1]['KO Loss SD']))/2
    fighter1_sub_score = (rnd.gauss(matchup_df.iloc[0]['Sub Wins Mean'],matchup_df.iloc[0]['Sub Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[1]['Sub Loss Mean'],matchup_df.iloc[1]['Sub Loss SD']))/2
    fighter2_dec_score = (rnd.gauss(matchup_df.iloc[1]['Decision Wins Mean'],matchup_df.iloc[1]['Decision Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[0]['Decision Loss Mean'],matchup_df.iloc[0]['Decision Loss SD']))/2
    fighter2_ko_score = (rnd.gauss(matchup_df.iloc[1]['KO Wins Mean'],matchup_df.iloc[1]['KO Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[0]['KO Loss Mean'],matchup_df.iloc[0]['KO Loss SD']))/2
    fighter2_sub_score = (rnd.gauss(matchup_df.iloc[1]['Sub Wins Mean'],matchup_df.iloc[1]['Sub Wins SD'])+ 
                          rnd.gauss(matchup_df.iloc[0]['Sub Loss Mean'],matchup_df.iloc[0]['Sub Loss SD']))/2   
    
    results.append(fighter1_dec_score)
    results.append(fighter1_ko_score)
    results.append(fighter1_sub_score)    
    results.append(fighter2_dec_score)    
    results.append(fighter2_ko_score)    
    results.append(fighter2_sub_score)
    
    
    if max(results) == results[0]:
        return "f1_dec"
    elif max(results) == results[1]:
        return "f1_ko"
    elif max(results) == results[2]:
        return "f1_sub"
    elif max(results) == results[3]:
        return "f2_dec"
    elif max(results) == results[4]:
        return "f2_ko"
    elif max(results) == results[5]:
        return "f2_sub"
    else: return "tie"

In [25]:
def gamesSim(ns):
    matchesout = []
    fighter1_decwin = 0
    fighter1_kowin = 0
    fighter1_subwin = 0
    fighter2_decwin = 0
    fighter2_kowin = 0
    fighter2_subwin = 0
    tie = 0
    for i in range(ns):
        gm = gameSim()
        matchesout.append(gm)
        if gm == "f1_dec":
            fighter1_decwin +=1 
        elif gm == "f1_ko":
            fighter1_kowin +=1 
        elif gm == "f1_sub":
            fighter1_subwin +=1 
        elif gm == "f2_dec":
            fighter2_decwin +=1 
        elif gm == "f2_ko":
            fighter2_kowin +=1 
        elif gm == "f2_sub":
            fighter2_subwin +=1 
        else: tie +=1 
    
    print(matchup_df.index[0] +' Decision Win ', round((fighter1_decwin/ns)*100,2),'%')
    print(matchup_df.index[0] +' KO Win ', round((fighter1_kowin/ns)*100,2),'%')
    print(matchup_df.index[0] +' Submission Win ', round((fighter1_subwin/ns)*100,2),'%')
    print(matchup_df.index[1] +' Decision Win ', round((fighter2_decwin/ns)*100,2),'%')
    print(matchup_df.index[1] +' KO Win ', round((fighter2_kowin/ns)*100,2),'%')
    print(matchup_df.index[1] +' Submission Win ', round((fighter2_subwin/ns)*100,2),'%')
    print('Tie ', (tie/ns)*100, '%')
    
    #return matchesout

## Predictions on UFC Fight Night: Pavlovich vs Blaydes

### Sergei Pavlovich vs Curtis Blaydes

In [32]:
matchup_df = get_fighter_parameters("Sergei Pavlovich", "Curtis Blaydes")
gamesSim(10000)
#sergei ko
#all correct

Sergei Pavlovich Decision Win  0.0 %
Sergei Pavlovich KO Win  92.76 %
Sergei Pavlovich Submission Win  0.0 %
Curtis Blaydes Decision Win  0.01 %
Curtis Blaydes KO Win  7.23 %
Curtis Blaydes Submission Win  0.0 %
Tie  0.0 %


### Brad Tavares vs Bruno Silva

In [33]:
matchup_df = get_fighter_parameters("Brad Tavares", "Bruno Silva")
gamesSim(10000)
#bruno silva ko
#all wrong
#2nd highest option correct

Brad Tavares Decision Win  81.73 %
Brad Tavares KO Win  0.0 %
Brad Tavares Submission Win  0.0 %
Bruno Silva Decision Win  0.0 %
Bruno Silva KO Win  18.27 %
Bruno Silva Submission Win  0.0 %
Tie  0.0 %


### Bobby Green vs Jared Gordon - No Contest

In [34]:
matchup_df = get_fighter_parameters("Bobby Green", "Jared Gordon")
gamesSim(10000)

Bobby Green Decision Win  4.91 %
Bobby Green KO Win  1.36 %
Bobby Green Submission Win  0.05 %
Jared Gordon Decision Win  93.65 %
Jared Gordon KO Win  0.03 %
Jared Gordon Submission Win  0.0 %
Tie  0.0 %


### Iasmin Lucindo vs Brogan Walker

In [35]:
matchup_df = get_fighter_parameters("Iasmin Lucindo", "Brogan Walker")
gamesSim(10000)
#iasmin decision
#winner correct, method wrong

Iasmin Lucindo Decision Win  0.0 %
Iasmin Lucindo KO Win  100.0 %
Iasmin Lucindo Submission Win  0.0 %
Brogan Walker Decision Win  0.0 %
Brogan Walker KO Win  0.0 %
Brogan Walker Submission Win  0.0 %
Tie  0.0 %


### Jeremiah Wells vs Matthew Semelsberger

In [36]:
matchup_df = get_fighter_parameters("Jeremiah Wells", "Matthew Semelsberger")
gamesSim(10000)
#jeremiah decision
#winner correct, method wrong

Jeremiah Wells Decision Win  3.67 %
Jeremiah Wells KO Win  62.95 %
Jeremiah Wells Submission Win  13.33 %
Matthew Semelsberger Decision Win  16.1 %
Matthew Semelsberger KO Win  3.95 %
Matthew Semelsberger Submission Win  0.0 %
Tie  0.0 %


### Rick Glenn vs Christos Giagos

In [37]:
matchup_df = get_fighter_parameters("Ricky Glenn", "Christos Giagos")
gamesSim(10000)
#christos ko
#winner correct, method wrong

Ricky Glenn Decision Win  37.98 %
Ricky Glenn KO Win  0.92 %
Ricky Glenn Submission Win  0.94 %
Christos Giagos Decision Win  59.98 %
Christos Giagos KO Win  0.0 %
Christos Giagos Submission Win  0.18 %
Tie  0.0 %


### Rani Yahya vs Montel Jackson

In [38]:
matchup_df = get_fighter_parameters("Rani Yahya", "Montel Jackson")
gamesSim(10000)
#montel ko 
#winner correct, method wrong

Rani Yahya Decision Win  15.25 %
Rani Yahya KO Win  0.0 %
Rani Yahya Submission Win  7.62 %
Montel Jackson Decision Win  76.09 %
Montel Jackson KO Win  1.04 %
Montel Jackson Submission Win  0.0 %
Tie  0.0 %


### Karol Rosa vs Norma Dumont

In [39]:
matchup_df = get_fighter_parameters("Karol Rosa", "Norma Dumont")
gamesSim(10000)
#norma decision
#all wrong
#2nd highest option was correct

Karol Rosa Decision Win  69.57 %
Karol Rosa KO Win  0.0 %
Karol Rosa Submission Win  0.0 %
Norma Dumont Decision Win  30.43 %
Norma Dumont KO Win  0.0 %
Norma Dumont Submission Win  0.0 %
Tie  0.0 %


### Mohammed Usman vs Junior Tafa - N/A (Junior Tafa's first fight recorded on fightmetric)

### Francis Marshall vs William Gomis

In [40]:
matchup_df = get_fighter_parameters("Francis Marshall", "William Gomis")
gamesSim(10000)
#william decision
#all correct

Francis Marshall Decision Win  7.37 %
Francis Marshall KO Win  7.19 %
Francis Marshall Submission Win  0.0 %
William Gomis Decision Win  85.44 %
William Gomis KO Win  0.0 %
William Gomis Submission Win  0.0 %
Tie  0.0 %


### Brady Hiestand vs Batgerel Danaa

In [41]:
matchup_df = get_fighter_parameters("Brady Hiestand", "Batgerel Danaa")
gamesSim(10000)
#brady ko
#winner correct, method wrong

Brady Hiestand Decision Win  64.42 %
Brady Hiestand KO Win  0.41 %
Brady Hiestand Submission Win  0.0 %
Batgerel Danaa Decision Win  21.09 %
Batgerel Danaa KO Win  14.08 %
Batgerel Danaa Submission Win  0.0 %
Tie  0.0 %


In [42]:
#2/9 perfect
#7/9 winner correct
#9/9 if including 2nd highest option

## Predictions on UFC Fight Night: Song vs Simón

### Song Yadong vs Ricky Simón

In [43]:
matchup_df = get_fighter_parameters("Song Yadong", "Ricky Simon")
gamesSim(10000)
#song ko
#all wrong
#2nd option was correct 

Song Yadong Decision Win  14.4 %
Song Yadong KO Win  32.58 %
Song Yadong Submission Win  0.01 %
Ricky Simon Decision Win  49.84 %
Ricky Simon KO Win  0.72 %
Ricky Simon Submission Win  2.45 %
Tie  0.0 %


### Caio Borralho vs Michal Oleksiejczuk

In [44]:
matchup_df = get_fighter_parameters("Caio Borralho", "Michal Oleksiejczuk")
gamesSim(10000)
#caio submission
#winner correct, method wrong 

Caio Borralho Decision Win  90.41 %
Caio Borralho KO Win  0.27 %
Caio Borralho Submission Win  0.09 %
Michal Oleksiejczuk Decision Win  0.0 %
Michal Oleksiejczuk KO Win  9.23 %
Michal Oleksiejczuk Submission Win  0.0 %
Tie  0.0 %


### Rodolfo Vieira vs Cody Brundage

In [45]:
matchup_df = get_fighter_parameters("Rodolfo Vieira", "Cody Brundage")
gamesSim(10000)
#rodolfo submission
#all correct

Rodolfo Vieira Decision Win  2.17 %
Rodolfo Vieira KO Win  17.03 %
Rodolfo Vieira Submission Win  56.44 %
Cody Brundage Decision Win  2.2 %
Cody Brundage KO Win  1.93 %
Cody Brundage Submission Win  20.23 %
Tie  0.0 %


### Julian Erosa vs Fernando Padilla - N/A (Fernando Padilla's first fight recorded on fightmetric)

### Marcos Rogério de Lima vs Waldo Cortes-Acosta

In [46]:
matchup_df = get_fighter_parameters("Marcos Rogerio de Lima", "Cody Brundage")
gamesSim(10000)
#marcos decision
#winner correct, method wrong

Marcos Rogerio de Lima Decision Win  5.84 %
Marcos Rogerio de Lima KO Win  68.71 %
Marcos Rogerio de Lima Submission Win  0.03 %
Cody Brundage Decision Win  0.0 %
Cody Brundage KO Win  1.63 %
Cody Brundage Submission Win  23.79 %
Tie  0.0 %


### Josh Quinlan vs Trey Waters

In [47]:
matchup_df = get_fighter_parameters("Josh Quinlan", "Trey Waters")
gamesSim(10000)
#trey decision
#all wrong 

Josh Quinlan Decision Win  0.0 %
Josh Quinlan KO Win  100.0 %
Josh Quinlan Submission Win  0.0 %
Trey Waters Decision Win  0.0 %
Trey Waters KO Win  0.0 %
Trey Waters Submission Win  0.0 %
Tie  0.0 %


### Martin Buday vs Jake Collier

In [48]:
matchup_df = get_fighter_parameters("Martin Buday", "Jake Collier")
gamesSim(10000)
#martin decision
#all correct

Martin Buday Decision Win  71.26 %
Martin Buday KO Win  28.33 %
Martin Buday Submission Win  0.0 %
Jake Collier Decision Win  0.41 %
Jake Collier KO Win  0.0 %
Jake Collier Submission Win  0.0 %
Tie  0.0 %


### Cody Durden vs Charles Johnson

In [49]:
matchup_df = get_fighter_parameters("Cody Durden", "Charles Johnson")
gamesSim(10000)
#cody decision
#all correct

Cody Durden Decision Win  85.07 %
Cody Durden KO Win  1.1 %
Cody Durden Submission Win  0.0 %
Charles Johnson Decision Win  2.77 %
Charles Johnson KO Win  2.62 %
Charles Johnson Submission Win  8.44 %
Tie  0.0 %


In [63]:
#3/7 perfect
#5/7 winner correct
#6/7 if 2nd outcome is accepted

### Stephanie Egger vs Irina Alekseeva - N/A (Irina Alekseeva first fight recorded on fightmetric)

### Journey Newson vs Marcus McGhee - N/A (Marcus McGhee first fight recorded on fightmetric)

### Hailey Cowan vs Jamey-Lyn Horth - N/A (Jamey-Lyn Horth first fight recorded on fightmetric)

## Predictions on UFC 288: Sterling vs. Cejudo

### Aljamain Sterling vs Henry Cejudo

In [51]:
matchup_df = get_fighter_parameters("Aljamain Sterling", "Henry Cejudo")
gamesSim(10000)

Aljamain Sterling Decision Win  29.21 %
Aljamain Sterling KO Win  0.16 %
Aljamain Sterling Submission Win  0.37 %
Henry Cejudo Decision Win  61.8 %
Henry Cejudo KO Win  8.46 %
Henry Cejudo Submission Win  0.0 %
Tie  0.0 %


### Belal Muhammad vs Gilbert Burns

In [52]:
matchup_df = get_fighter_parameters("Belal Muhammad", "Gilbert Burns")
gamesSim(10000)

Belal Muhammad Decision Win  93.44 %
Belal Muhammad KO Win  0.04 %
Belal Muhammad Submission Win  0.0 %
Gilbert Burns Decision Win  6.41 %
Gilbert Burns KO Win  0.06 %
Gilbert Burns Submission Win  0.05 %
Tie  0.0 %


### Jessica Andrade vs Yan Xiaonan

In [53]:
matchup_df = get_fighter_parameters("Jessica Andrade", "Yan Xiaonan")
gamesSim(10000)

Jessica Andrade Decision Win  1.52 %
Jessica Andrade KO Win  0.34 %
Jessica Andrade Submission Win  0.0 %
Yan Xiaonan Decision Win  98.14 %
Yan Xiaonan KO Win  0.0 %
Yan Xiaonan Submission Win  0.0 %
Tie  0.0 %


### Movsar Evloev vs Bryce Mitchell

In [54]:
matchup_df = get_fighter_parameters("Movsar Evloev", "Bryce Mitchell")
gamesSim(10000)

Movsar Evloev Decision Win  95.15 %
Movsar Evloev KO Win  0.0 %
Movsar Evloev Submission Win  0.0 %
Bryce Mitchell Decision Win  4.85 %
Bryce Mitchell KO Win  0.0 %
Bryce Mitchell Submission Win  0.0 %
Tie  0.0 %


### Kron Gracie vs Charles Jourdain

In [55]:
matchup_df = get_fighter_parameters("Kron Gracie", "Charles Jourdain")
gamesSim(10000)

Kron Gracie Decision Win  13.28 %
Kron Gracie KO Win  0.0 %
Kron Gracie Submission Win  42.52 %
Charles Jourdain Decision Win  42.94 %
Charles Jourdain KO Win  1.19 %
Charles Jourdain Submission Win  0.07 %
Tie  0.0 %


### Drew Dober vs Matt Frevola

In [26]:
matchup_df = get_fighter_parameters("Drew Dober", "Matt Frevola")
gamesSim(10000)

Drew Dober Decision Win  1.7 %
Drew Dober KO Win  82.82 %
Drew Dober Submission Win  0.0 %
Matt Frevola Decision Win  10.63 %
Matt Frevola KO Win  1.39 %
Matt Frevola Submission Win  3.46 %
Tie  0.0 %


### Kennedy Nzechukwu vs Devin Clark

In [57]:
matchup_df = get_fighter_parameters("Kennedy Nzechukwu", "Devin Clark")
gamesSim(10000)

Kennedy Nzechukwu Decision Win  0.98 %
Kennedy Nzechukwu KO Win  69.69 %
Kennedy Nzechukwu Submission Win  0.08 %
Devin Clark Decision Win  29.14 %
Devin Clark KO Win  0.1 %
Devin Clark Submission Win  0.01 %
Tie  0.0 %


### Khaos Williams vs Rolando Bedoya - N/A (Rolando Bedoya first fight recorded on fightmetric)

### Marina Rodriguez vs Virna Jandiroba

In [58]:
matchup_df = get_fighter_parameters("Marina Rodriguez", "Virna Jandiroba")
gamesSim(10000)

Marina Rodriguez Decision Win  97.63 %
Marina Rodriguez KO Win  0.21 %
Marina Rodriguez Submission Win  0.0 %
Virna Jandiroba Decision Win  0.67 %
Virna Jandiroba KO Win  0.54 %
Virna Jandiroba Submission Win  0.95 %
Tie  0.0 %


### Braxton Smith vs Parker Porter - N/A (Braxton Smith first fight recorded on fightmetric)

### Phil Hawes vs Ikram Aliskerov

In [59]:
matchup_df = get_fighter_parameters("Phil Hawes", "Ikram Aliskerov")
gamesSim(10000)

Phil Hawes Decision Win  0.0 %
Phil Hawes KO Win  0.01 %
Phil Hawes Submission Win  0.0 %
Ikram Aliskerov Decision Win  0.0 %
Ikram Aliskerov KO Win  0.02 %
Ikram Aliskerov Submission Win  99.97 %
Tie  0.0 %


### Rafael Estevam vs Zhalgas Zhumagulov

In [60]:
matchup_df = get_fighter_parameters("Rafael Estevam", "Zhalgas Zhumagulov")
gamesSim(10000)

Rafael Estevam Decision Win  2.01 %
Rafael Estevam KO Win  97.99 %
Rafael Estevam Submission Win  0.0 %
Zhalgas Zhumagulov Decision Win  0.0 %
Zhalgas Zhumagulov KO Win  0.0 %
Zhalgas Zhumagulov Submission Win  0.0 %
Tie  0.0 %


### Joseph Holmes vs Claudio Ribeiro

In [61]:
matchup_df = get_fighter_parameters("Joseph Holmes", "Claudio Ribeiro")
gamesSim(10000)

Joseph Holmes Decision Win  0.0 %
Joseph Holmes KO Win  31.47 %
Joseph Holmes Submission Win  26.69 %
Claudio Ribeiro Decision Win  4.78 %
Claudio Ribeiro KO Win  32.76 %
Claudio Ribeiro Submission Win  4.3 %
Tie  0.0 %


### Daniel Santos vs Johnny Munoz

In [62]:
matchup_df = get_fighter_parameters("Daniel Santos", "Johnny Munoz")
gamesSim(10000)

Daniel Santos Decision Win  2.77 %
Daniel Santos KO Win  46.71 %
Daniel Santos Submission Win  0.0 %
Johnny Munoz Decision Win  47.81 %
Johnny Munoz KO Win  0.0 %
Johnny Munoz Submission Win  2.71 %
Tie  0.0 %
